# Object detection

In [ ]:
# !mim download mmdet --config mask-rcnn_swin-t-p4-w7_fpn_1x_coco --dest .
%load_ext autoreload
%autoreload 2

from PIL import Image
import torch

from mmdet.apis import init_detector, inference_detector
from mmengine.config import Config
from cfm_task_models.legacy import *

from cfm_task_models.split_utils import SplitSwinTransformer, SplitTwoStageDetector, TwoInputIdentity, SplitRepPointsV2MaskDetector

config_file_old = 'obj_det/cfgs/swin_tiny_mask_rcnn_simplified_cfg.py'
checkpoint_file_old = 'obj_det/chkpts/mask_rcnn_swin-t-p4-w7_fpn_1x_coco_20210902_120937-9d6b7cfa.pth'
config_file = 'obj_det/cfgs/mask_reppointsv2_swin_t_modified.py'
checkpoint_file = 'obj_det/chkpts/mask_reppointsv2_swin_tiny_patch4_window7_3x.pth'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model_init = init_detector(config_file, checkpoint_file, device='cpu')  # or device='cuda:0'
# result = inference_detector(model, 'demo/demo.jpg')

# from mmdet.registry import MODELS
# MODELS.get('SwinTransformer')
# MODELS.get('TwoStageDetector')
model = SplitRepPointsV2MaskDetector.create_from_cfg_and_checkpoint(config_file, checkpoint_file).to(device)
# model2 = SplitTwoStageDetector.create_from_cfg_and_checkpoint(config_file_old,checkpoint_file_old).to(device)
# model.frontend_preprocessor

In [ ]:
from mmengine.runner import Runner
from mmdet.structures import DetDataSample
from typing import Union
from torch import Tensor
def to_device(data_sample:Union[dict,Tensor], device):
    if isinstance(data_sample,dict):
        for k,v in data_sample.items():
            if isinstance(v, (Tensor, DetDataSample)):
                data_sample[k] = v.to(device)
                # print(f'{k} was sent to {device}')
            elif isinstance(v, list):
                new_v = [to_device(v2, device) for v2 in v]
                data_sample[k] = new_v
                # print(f'components of {k} were sent to {device}')
            else:
                print(f'{k} could not be sent to {device}')
    elif isinstance(data_sample, (Tensor, DetDataSample)):
        data_sample = data_sample.to(device)
        # print(f'input was sent to {device}')
    elif isinstance(data_sample, list):
        data_sample = [to_device(v, device) for v in data_sample]
        # print(f'components of input were sent to {device}')
    else:
        # print(type(data_sample))
        print(f'input could not be sent to {device}')
    return data_sample


In [ ]:


cfg_tr = model.cfg.copy()
print(cfg_tr['train_dataloader']['dataset']['ann_file'])
cfg_tr['train_dataloader']['batch_size'] = 1
cfg_tr['train_dataloader']['num_workers'] = 1

train_dataloader = Runner.build_dataloader(cfg_tr['train_dataloader']) 

print(cfg_tr['val_dataloader']['dataset']['ann_file'])
val_dataloader = Runner.build_dataloader(cfg_tr['val_dataloader'])

test_dataloader = Runner.build_dataloader(cfg_tr['test_dataloader'])
print(len(val_dataloader.dataset))

In [ ]:
# dl = runner.train_dataloader
# print(model.cut_point)
# val_eval = Runner.build_evaluator(_,cfg_tr['val_evaluator'])
# setattr(val_eval,'dataset_meta',val_dataloader.dataset.metainfo)
from copy import deepcopy
test_evaluator_cfg = deepcopy(cfg_tr['test_evaluator'])
print(cfg_tr['train_dataloader']['dataset']['data_root'],cfg_tr['test_dataloader']['dataset']['ann_file'] )
test_evaluator_cfg['ann_file'] = os.path.join(cfg_tr['test_dataloader']['dataset']['data_root'], test_evaluator_cfg['ann_file'])
# # print(test_evaluator_cfg)
test_eval = Runner.build_evaluator(_,cfg_tr['test_evaluator'])
setattr(test_eval,'dataset_meta',test_dataloader.dataset.metainfo)
# print(val_eval.dataset_meta)
for i,data in enumerate(train_dataloader):
    model.train()
    model.zero_grad()

    # data =  to_device(data,device)
    # feat = model2.feature_frontend(data)
    # print(feat['inputs']['outs'][0].shape)
    # out2 = model2.test_step(feat)
    # print(data['data_samples'][0])
    if isinstance(model, SplitRepPointsV2MaskDetector):
        if 'scale_factor' in data['data_samples'][0]:
            for d in data['data_samples']:
                meta = d.metainfo 
                # print(d)
                meta['scale_factor'] = d.scale_factor * (2 if len(meta['scale_factor']) == 2 else 1)
                d.set_metainfo(meta)

    # im['inputs'] = torch.stack(im['inputs'])
    # print(list(data_.keys()))
    # data_ = train_pipeline(im)
    # data_ = prp(data_, False)
    # data_['inputs'] = model.backbone.split_forward_v2(data_['inputs'], output_layer=model.cut_point-1)
    # data_['inputs'] = {
    #                "hw_shape": data_['inputs'][0],
    #
    #                "outs": data_['inputs'][1]}
    # data = prp(data, False)

    
    feat = model.feature_frontend(data)
    # print(feat['inputs']['outs'][0].shape)
    loss = model.backend_loss(feat)
    if i > 100:
        break
    # out = model.test_step(feat)
    # props = model.backend_raw(feat)
    # print(model.cut_point)
    # print(model.test_step(feat)[0])
    # losses = model.loss(data_['inputs'], data_['data_samples'])
    # loss, losses = model.parse_losses(losses)
    # test_eval.process(out)
# print(out[0].pred_instances.masks)
# test_eval.evaluate(i+1)


for i,data in enumerate(test_dataloader):
    model.eval()
    model.zero_grad()
    with torch.no_grad():
    
        # data =  to_device(data,device)
        # feat = model2.feature_frontend(data)
        # print(feat['inputs']['outs'][0].shape)
        # out2 = model2.test_step(feat)
        if isinstance(model, SplitRepPointsV2MaskDetector):
            for d in data['data_samples']:
                meta = d.metainfo 
                meta['scale_factor'] = d.scale_factor * (2 if len(meta['scale_factor']) == 2 else 1)
                d.set_metainfo(meta)

        # im['inputs'] = torch.stack(im['inputs'])
        # print(list(data_.keys()))
        # data_ = train_pipeline(im)
        # data_ = prp(data_, False)
        # data_['inputs'] = model.backbone.split_forward_v2(data_['inputs'], output_layer=model.cut_point-1)
        # data_['inputs'] = {
        #                "hw_shape": data_['inputs'][0],
        #
        #                "outs": data_['inputs'][1]}
        # data = prp(data, False)

        
        feat = model.feature_frontend(data)
        # print(feat['inputs']['outs'][0].shape)
        out = model.test_step(feat)
        # props = model.backend_raw(feat)
        # print(model.cut_point)
        # print(model.test_step(feat)[0])
        # losses = model.loss(data_['inputs'], data_['data_samples'])
        # loss, losses = model.parse_losses(losses)
        test_eval.process(out)
# print(out[0].pred_instances.masks)
test_eval.evaluate(i+1)

In [ ]:
torch.int

In [ ]:
val_eval = Runner.build_evaluator(_,cfg_tr['val_evaluator'])
setattr(val_eval,'dataset_meta',val_dataloader.dataset.metainfo)

# test_evaluator_cfg = deepcopy(cfg_tr['test_evaluator'])
# print(train_dataset_cfg['data_root'],cfg_tr['test_dataloader']['dataset']['ann_file'] )
# test_evaluator_cfg['ann_file'] = os.path.join(cfg_tr['test_dataloader']['dataset']['data_root'], test_evaluator_cfg['ann_file'])
# print(test_evaluator_cfg)
test_eval = Runner.build_evaluator(_,cfg_tr['test_evaluator'])
setattr(test_eval,'dataset_meta',test_dataloader.dataset.metainfo)
# print(val_eval.dataset_meta)
# print(out)
test_eval.process(out)
test_eval.evaluate(1)
print(test_eval.metrics[0].results)

In [ ]:
a = Instances()

fk = torch.Tensor([[[3,4]],[[4,5]],[[5,6]]])
device = 'cuda'
a.fpn_levels = torch.cat([torch.tensor([i]*r[:,0].numel()).to(device) for i,r in enumerate(fk)])

In [ ]:
from matplotlib import pyplot as plt
mx = feat['inputs']['outs'][0][0,:].max().item()
mn = feat['inputs']['outs'][0][0,:].min().item()
f_np = feat['inputs']['outs'][0][0,:].reshape(2048,-1).cpu().detach()
f_np = f_np.detach().numpy()
# print(f_np)
plt.hist((f_np-mn)/(mx-mn))

In [ ]:
# feat2 = feat['inputs']['outs'][0].view(-1, *feat['inputs']['hw_shape'],
#                                 model.backbone.num_features[0]).permute(0, 3, 1,
#                                                                 2).contiguous()
# print(feat2.shape)
norm0 = getattr(model.backbone, 'norm0')
feats_to_comp  = norm0( feat['inputs']['outs'][0]).view(-1, *feat['inputs']['hw_shape'],
                                model.backbone.num_features[0]).permute(0, 3, 1,
                                                                2).contiguous()
print(feats_to_comp.shape)

In [ ]:
for data in vdl:
    # model.cut_point = 0
    # model.data_preprocessor = model.frontend_preprocessor
    # print(model.val_step(data))
    # model.cut_point = 1
    # model.data_preprocessor = TwoInputIdentity()
    data_ = model.frontend_preprocessor(data, False)
    print(data_['data_samples'])
    break
    data_['inputs'] = model.backbone.split_forward_v2(data_['inputs'], output_layer=model.cut_point-1)
    data_['inputs'] = {#"x": data_['inputs'][0],
                    "hw_shape": data_['inputs'][0],
                    "outs": data_['inputs'][1]}
    print(len(data_['inputs']['outs']))
    print(data_)
    # data_['inputs'] = [data_['inputs']]
    # data_['data_samples'] = [data_['data_samples']]
    # # forward the model
    with torch.no_grad():
        results = model.backend_inference(data_)[0]
    break

In [ ]:
vdl = runner.val_dataloader

In [ ]:
model.backbone.stages[0]

In [ ]:
from numpy import log2 as log
def H(d):
    return -d*log(d) + (d-1)*log(1-d)

m = 8
D = np.linspace(0.1442,0.14421,10000000)
# R = 1-D*m/(m-1)
# rate = R*log(m)
the_rate = log(m) -H(D) - D * log(m-1) - 2
print(D[np.where(the_rate < 0)[0][0]])
the_rate[np.where(the_rate < 0)[0][0]] + 2

# Semantic Segmentation

In [ ]:
%load_ext autoreload
%autoreload 2

from mmseg.apis import inference_model, init_model, show_result_pyplot
import mmcv

from PIL import Image
import torch

from mmengine.config import Config
from mmengine.dataset import Compose

from split_utils import SplitEncoderDecoder, TwoInputIdentity

config_file = "../../segmentation/maskformer_swin-t_upernet_8xb2-160k_ade20k-512x512_modified.py"
checkpoint_file = "../../segmentation/maskformer_swin-t_upernet_8xb2-160k_ade20k-512x512_20221114_232813-f14e7ce0.pth"


# Build the model from a config file and a checkpoint file
model_init = init_model(config_file, checkpoint_file, device='cpu') # or device='cuda:0'

# test a single image and show the results
img = '../../demo/demo.jpg'  # or img = mmcv.imread(img), which will only load it once
result = inference_model(model_init, img)
# # visualize the results in a new window
# # show_result_pyplot(model, img, result, show=True)
# # or save the visualization results to image files
# # you can change the opacity of the painted segmentation map in (0, 1].
# show_result_pyplot(model, img, result, show=True, out_file='result.jpg', opacity=0.5)

In [ ]:
%load_ext autoreload
%autoreload 2

from mmengine.config import Config
from mmengine.runner import Runner
from split_utils import SplitEncoderDecoder #, get_mean_metrics


In [ ]:
# setup the config file
cfg = Config.fromfile('../../segmentation/swin-tiny-patch4-window7-in1k-pre_upernet_8xb2-160k_ade20k-512x512.py')

# set the data_root of the config file
data_root = '../../data/ade/ADEChallengeData2016'
cfg.data_root = data_root
cfg.test_dataloader.dataset.data_root = data_root
cfg.train_dataloader.dataset.data_root = data_root
cfg.val_dataloader.dataset.data_root = data_root


train_pipeline = [{'type': 'LoadImageFromFile'},
                  {'reduce_zero_label': True, 'type': 'LoadAnnotations'},
                  {'keep_ratio': True,
                   'ratio_range': (0.5, 2.0),
                   'scale': (2048, 1024), # (2048, 512)
                   'type': 'RandomResize'},
                   {'cat_max_ratio': 0.75, 'crop_size': (512, 512), 'type': 'RandomCrop'},
                   {'prob': 0.5, 'type': 'RandomFlip'},
                   {'type': 'PhotoMetricDistortion'},
                   {'type': 'PackSegInputs'}]

cfg.train_dataloader.dataset.pipeline = train_pipeline

cfg.dump('../../segmentation/swin-tiny-patch4-window7-in1k-pre_upernet_8xb2-160k_ade20k-512x512_modified.py')

config_file = "../../segmentation/swin-tiny-patch4-window7-in1k-pre_upernet_8xb2-160k_ade20k-512x512_modified.py"
checkpoint_file = '../../segmentation/upernet_swin_tiny_patch4_window7_512x512_160k_ade20k_pretrain_224x224_1K_20210531_112542-e380ad3e.pth'


In [ ]:
# config_file = "../../segmentation/swin-tiny-patch4-window7-in1k-pre_upernet_8xb2-160k_ade20k-512x512_modified.py"
# checkpoint_file = '../../segmentation/upernet_swin_tiny_patch4_window7_512x512_160k_ade20k_pretrain_224x224_1K_20210531_112542-e380ad3e.pth'

# model = SplitEncoderDecoder.create_from_cfg_and_checkpoint(config_file, checkpoint_file)

# cfg['work_dir'] = './logs'
# runner = Runner.from_cfg(cfg)

In [ ]:
# model = SplitEncoderDecoder.create_from_cfg_and_checkpoint(config_file, checkpoint_file)
# dataloader = Runner.build_dataloader(cfg.val_dataloader)

# cut point = 0, 1 test


model = SplitEncoderDecoder.create_from_cfg_and_checkpoint(config_file, checkpoint_file)
# model.set_cut_point(0) # so that input_layer=self.cut_point=0
# model.swap_preprocessor()
model.zero_grad()
model.eval()
model.to('cuda')

dataloader = Runner.build_dataloader(cfg.val_dataloader)
val_eval = Runner.build_evaluator(_, cfg.val_evaluator)
setattr(val_eval,'dataset_meta', dataloader.dataset.metainfo)


    
# eval_results = []
for i, data in enumerate(dataloader):
    print(f"Processing image {i}")
    features = model.feature_frontend(data)
    result_backend_inference = model.backend_inference(features)
    val_eval.process(result_backend_inference)
    # eval_result = val_eval.offline_evaluate(result_backend_inference)
    # eval_results.append(eval_result)
    # result_backend_raw = model.backend_raw(data)
    print(f"Processed image {i}")
    if i > 2:
        break

In [ ]:
result = val_eval.evaluate(len(dataloader))
# result = val_eval.offline_evaluate(result_backend_inference)

In [ ]:
# import cv2
# img_path = "/home/data/ade/ADEChallengeData2016/images/validation/ADE_val_00000001.jpg"
# img = cv2.imread(img_path)
# cv2.imshow("image", img)

## Check SplitEncoderDecoder

In [ ]:
import torch
import json
import os

from tqdm import tqdm
import matplotlib.pyplot as plt

from mmengine.config import Config
from mmengine.runner import Runner
from split_utils import SplitEncoderDecoder, set_data_root_in_cfg



In [ ]:
def show_img(data):
    plt.imshow(data['inputs'][0].permute(1,2,0).cpu().numpy())

In [ ]:
config = "../configs/segmentation/mask2former_swin-t_8xb2-160k_ade20k-512x512.py"
checkpoint = "../checkpoints/mask2former_swin-t_8xb2-160k_ade20k-512x512_20221203_234230-7d64e5dd.pth"
data_root = "/home/data/ade/ADEChallengeData2016"

cut_point = 0

cfg = Config.fromfile(config)
set_data_root_in_cfg(cfg, data_root)

model = SplitEncoderDecoder.create_from_cfg_and_checkpoint(config, checkpoint, cut_point=cut_point)
model.eval()

In [ ]:
# cfg.train_dataloader.sampler.shuffle = False
# cfg.train_dataloader.sampler.type='DefaultSampler'
dataloader = Runner.build_dataloader(cfg.val_dataloader)
print(len(dataloader))

In [ ]:
for data in dataloader:
    features = model.feature_frontend(data)
    result = model.backend_inference(features)
    print(result)
    break

In [ ]:
for data in dataloader:
    # show_img(data)
    
    features = model.feature_frontend(data)
    loss = model.backend_loss(features)
    print(loss)
    break

In [ ]:
for data in dataloader:
    
    data = model.frontend_preprocessor(data)
    result = model.predict(data['inputs'], data['data_samples'])
    print(result)
    break

In [ ]:
print(loss)

In [ ]:
features

In [ ]:
features['inputs']['outs'][0].shape